## 직선 학습하기

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston

In [3]:
boston=load_boston()
features=boston.data[:,0:2]
target=boston.target

In [12]:
regression=LinearRegression()

In [13]:
model=regression.fit(features,target)

선형회귀는 특성과 타깃 벡터 사이의 관계가 거의 선형이라고 가정한다.

즉 타깃 벡터에 대한 특성의 효과(계수, 가중치, 파라미터 등)은 상수이다.

In [14]:
# 모델의 절편

model.intercept_

22.485628113468223

In [15]:
# 모델 특성 계수

model.coef_

array([-0.35207832,  0.11610909])

In [16]:
'''
데이터셋에서 타깃값은 1970년대 보스턴 주택의 중간 가격으로 천달러 단위이다.
데이터셋에 있는 첫번째 주택 가격은 다음과 같다
'''

target[0]*1000

24000.0

In [17]:
# predict 메소드를 사용하여 주택값을 예측할 수 있다.

model.predict(features)[0]*1000

24573.366631705547

선형 회귀의 주요 장점은 해석이 용이하다는 것이다.

모델의 계수는 특성이 타깃값에 미치는 영향의 크기이다.

In [19]:
# 1번째 특성인 인당 범죄율이 1 증가할때마다 집값에 미치는 영향 살펴보기

model.coef_[0]*1000

-352.07831564026765

인구당 범죄율이 1 증가할때마다 집값이 약 $350 정도 감소함을 알 수 있다.

## 교차 특성 다루기
### 타깃 변수에 영향을 미치면서 다른 특성에 의존하는 특성의 의존성 잡기

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures

In [21]:
boston=load_boston()
features=boston.data[:,0:2]
target=boston.target

In [24]:
# 교차항 생성

interaction=PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)
features_interaction=interaction.fit_transform(features)

In [25]:
# 선형 회귀모델 생성

reg=LinearRegression()

In [27]:
# 선형 회귀모델 훈련

model=reg.fit(features_interaction, target)

교차항을 만드는 까닭은, 각 특성이 독립적으로 존재할때 타깃에 미치는 영향보다 함께 있음으로써 미치는 영향이 클 경우를 고려하기 때문이다

In [28]:
# 첫번째 특성 확인

features[0]

array([6.32e-03, 1.80e+01])

In [29]:
import numpy as np

# 각 샘플에서 첫번째와 두번째 특성을 곱한다.

interaction_term=np.multiply(features[:,0], features[:,1])

In [30]:
# 첫번째 샘플의 교차항 확인

interaction_term[0]

0.11376

PolynomialFeatures 의 매개변수

- interaction_only : True 로 지정할 경우 교차항만 반환
- include_bias : False 로 지정할 경우 절편(bias) 라고 불리는 1 값을 추가하지 않음
- degree : 교차항을 만들 최대 특성의 수를 결정한다.

In [31]:
# PolynomialFeatures 를 사용했을 때의 첫번째 교차항 확인

features_interaction[0]

array([6.3200e-03, 1.8000e+01, 1.1376e-01])

## 비선형 관계 학습

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures

In [33]:
boston=load_boston()
features=boston.data[:,0:1]
target=boston.target

In [34]:
# 다항특성 x^2, x^3  생성

polynomial=PolynomialFeatures(degree=3, include_bias=False)
features_poly=polynomial.fit_transform(features)

In [35]:
# 선형 회귀 모델을 만든다.

reg=LinearRegression()

In [36]:
# 모델 훈련

model=reg.fit(features_poly, target)

In [38]:
print(features[0]," 첫번째 샘플 값")
print(features[0]**2," 첫번째 샘플의 거듭제곱 값")
print(features[0]**3," 첫번째 샘플의 세제곱 값")

[0.00632]  첫번째 샘플 값
[3.99424e-05]  첫번째 샘플의 거듭제곱 값
[2.52435968e-07]  첫번째 샘플의 세제곱 값


In [39]:
# PolynomialFeatures 결과

features_poly[0]

array([6.32000000e-03, 3.99424000e-05, 2.52435968e-07])

## 규제로 분산 줄이기
### 선형 회귀의 분산을 줄이는 방법 --> 리지/라소 같은 축소 페널티(규제) 가 포함된 학습 알고리즘 사용

In [41]:
from sklearn.linear_model import Ridge
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

In [43]:
boston=load_boston()
features=boston.data
target=boston.target

In [44]:
# 특성 표준화

scaler=StandardScaler()
features_standard=scaler.fit_transform(features)

In [45]:
# alpha 값을 지정한 리지 회귀 생성

regression=Ridge(alpha=0.5)

In [46]:
# 선형 회귀모델 훈련

model=regression.fit(features_standard, target)

리지 회귀에서 규제는 모든 계수의 제곱합에 튜닝 파라미터를 곱한것이다.

라소 회귀에서 규제는 모든 계수의 절대값에 튜닝 파라미터를 곱한것이다.

alpha 값이 클수록 더 간단한 모델을 구성한다.

좋은 alpha 값을 선택하기 위해서 RidgeCV 같은 클래스를 사용한다.

In [47]:
from sklearn.linear_model import RidgeCV

In [48]:
# 세 개의 alpha 값에 대한 리지 회귀 생성

reg_cv=RidgeCV(alphas=[0.1,1.0,10.0])

In [49]:
# 선형 회귀모델 훈련

model_cv=reg_cv.fit(features_standard, target)

In [50]:
# 계수 확인

model_cv.coef_

array([-0.91987132,  1.06646104,  0.11738487,  0.68512693, -2.02901013,
        2.68275376,  0.01315848, -3.07733968,  2.59153764, -2.0105579 ,
       -2.05238455,  0.84884839, -3.73066646])

In [51]:
# 최선의 모델이 사용한 alpha 값 확인

model_cv.alpha_

1.0

RidgeCV 클래스의 cv 매개변수를 사용하여 교차검증 방식을 지정할 수 있다.

Default는 None 으로 리브원아웃 방식을 사용하고, 정수를 지정하면 GridSearchCV를 사용하여 교차검증을 진행한다

In [52]:
# 5폴드 교차검증 사용하여 리지 회귀 만들기

reg_cv=RidgeCV(alphas=[0.1,1.0,10.0], cv=5)

In [53]:
# 선형 회귀모델 훈련
model_cv=reg_cv.fit(features_standard, target)

In [54]:
# alpha 값 확인

model_cv.alpha_

10.0

## 라쏘 회귀로 특성 줄이기
### 특성의 수를 줄여서 선형 회귀모델을 단순하게 구성

In [55]:
from sklearn.linear_model import Lasso
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

In [56]:
# 데이터 로드

boston=load_boston()
features=boston.data
target=boston.target

In [57]:
# 특성 표준화

scaler=StandardScaler()
features_standard=scaler.fit_transform(features)

In [58]:
# alpha 를 지정한 Lasso 회귀모델 생성

reg=Lasso(alpha=0.5)

In [59]:
# 모델 훈련

model=reg.fit(features_standard, target)

라쏘 회귀모델의 특징은 모델의 계수를 0까지 줄일 수 있다는 것이다.

이를 이용하여 특성의 수를 줄일 수 있다.

In [60]:
model.coef_

array([-0.11526463,  0.        , -0.        ,  0.39707879, -0.        ,
        2.97425861, -0.        , -0.17056942, -0.        , -0.        ,
       -1.59844856,  0.54313871, -3.66614361])

In [61]:
''' alpha 값을 너무 크게 지정하면 어떤 특성도 사용하지 않을 수도 있다'''

reg_a10=Lasso(alpha=10)
model_a10=reg_a10.fit(features_standard, target)
model_a10.coef_

array([-0.,  0., -0.,  0., -0.,  0., -0.,  0., -0., -0., -0.,  0., -0.])

Lasso 의 alpha 값을 찾기위해 LassoCV 클래스를 사용할 수 있다

In [62]:
from sklearn.linear_model import LassoCV

In [63]:
# 세 개의 alpha 값에 대한 라쏘 회귀 생성

lasso_cv=LassoCV(alphas=[0.1,1.0,10.0], cv=5)

In [64]:
# 회귀 모델 훈련

model_cv=lasso_cv.fit(features_standard, target)

In [65]:
# 계수 확인

model_cv.coef_

array([-0.63230364,  0.70840931, -0.        ,  0.65760723, -1.57419335,
        2.82626903, -0.        , -2.42207901,  1.19593681, -0.84646778,
       -1.92249345,  0.76216539, -3.72618383])

In [66]:
# 최선의 모델을 위한 alpha 값 확인

model_cv.alpha_

0.1

In [67]:
'''
LassoCV는 alphas 매개변수에 탐색할 값을 명시적으로 정하지 않고 n_alphas 매개변수를 사용해 자동으로 탐색 대상 값을 생성할 수 있다.
n_alphas의 기본값은 100으로, 탐색후보로 100개의 값을 선정한다
'''

lasso_cv=LassoCV(n_alphas=1000, cv=5)

In [68]:
# 선형 회귀모델 훈련

model_cv=lasso_cv.fit(features_standard, target)

In [69]:
# 계수확인

model_cv.coef_

array([-0.47509958,  0.51087329, -0.06951944,  0.64416411, -1.32325523,
        2.90601026, -0.        , -2.05439547,  0.42015063, -0.19614605,
       -1.85051983,  0.71550534, -3.72013448])

In [70]:
# alpha 값 확인

model_cv.alpha_

0.1532617308309083